In [ ]:
from google.colab import drive
import os, io, cv2, base64, requests, json, time, sys

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Converting img folder into img filenames.txt pt 1
import os

folder_path='/content/drive/MyDrive/Final Project/bbcnews'

filenames = os.listdir(folder_path)
print(filenames)

In [ ]:
#Converting img folder into img filenames.txt pt 2
output_file_path = '/content/drive/MyDrive/Final Project/bbc img filenames.txt'

with open(output_file_path, 'w') as f:
    for name in filenames:
        f.write(name + '\n')

print("Filenames have been saved to:", output_file_path)


In [ ]:
#Converting img folder into img filenames.txt pt 3
imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
print(imgnamefilepath)
imgnamefile = open(imgnamefilepath, "r")

for i, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip("\r\n")
    print(i, imgname)

In [ ]:
#Face ++ multiple Face Analysis
API_KEY = "GFnZ6fljq23SWOe-Sq5TfswFtsxl14h0"
API_SECRET = "Zx-pEmLroe-4QEFjImmLb6MEHfebPdz9"

http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'

attributes="gender,age,emotion"

imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
print(imgnamefilepath)

imgnamefile = open(imgnamefilepath, "r")

for i, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip("\r\n")
    print(i, imgname)


    imgpath = os.path.join('/content/drive/MyDrive',"Final Project", "bbcnews", imgname)

    if len(imgpath) > 255:
        imgpath = "/".join(imgpath.split("/")[1:])
    image_file = open(imgpath, "rb")
    encoded_string = base64.b64encode(image_file.read())


    r = requests.post(http_url,  data = {'api_key': API_KEY,
                             'api_secret': API_SECRET,
                             'image_base64': encoded_string,
                             'return_landmark': 0,
                             'return_attributes': attributes})

    print(r.text)

    js = json.loads(r.text)
    print(json.dumps(js, indent=4))


    faces = js["faces"]
    numface = len(faces)
    print("number of faces",numface)

    if numface == 0:

        wlist = [imgname, numface]
        print(wlist)

        slist = [str(x) for x in wlist]
        print(slist)

        savelist = "\t".join(slist)
        resultpath = os.path.join('/content/drive/MyDrive',"Final Project", "bbcnews results.txt")


        with open(resultpath, 'a') as file:
            file.write(savelist + '\n')

    for faceid in range(0, numface):
        if faceid > 4: continue
        print("face id", faceid)
        this_face = faces[faceid]
        width = this_face["face_rectangle"]["width"]; height = this_face["face_rectangle"]["height"]
        top = this_face["face_rectangle"]["top"]; left = this_face["face_rectangle"]["left"]
        size = width * height
        print("face size",size)
        gender = this_face["attributes"]["gender"]["value"]
        age = this_face["attributes"]["age"]["value"]
        anger = this_face["attributes"]["emotion"]["anger"]
        disgust = this_face["attributes"]["emotion"]["disgust"]
        fear = this_face["attributes"]["emotion"]["fear"]
        happiness = this_face["attributes"]["emotion"]["happiness"]
        neutral = this_face["attributes"]["emotion"]["neutral"]
        sadness = this_face["attributes"]["emotion"]["sadness"]
        surprise = this_face["attributes"]["emotion"]["surprise"]

        print("gender",gender)
        print("age",age)
        print("emotion",anger, disgust, fear, happiness, neutral, sadness, surprise)


        wlist = [imgname, numface, faceid, gender, age, anger, disgust, fear, happiness, neutral, sadness, surprise]
        print(wlist)

        slist = [str(x) for x in wlist]
        print(slist)

        savelist = "\t".join(slist)
        resultpath = os.path.join('/content/drive/MyDrive',"Final Project", "bbcnews results.txt")

        with open(resultpath, 'a') as file:
            file.write(savelist + '\n')


    time.sleep(1)

In [ ]:
#Running an Aesthetic Analysis
import os, sys, cv2
import numpy as np

imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
print(imgnamefilepath)

imgnamefile = open(imgnamefilepath, "r")

for i, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip("\r\n")
    print(i, imgname)

    # create the image path
    imgpath = os.path.join('/content/drive/MyDrive',"Final Project", "bbcnews", imgname)

    # get the file size
    filesize = os.path.getsize(imgpath)
    print(filesize)

    img = cv2.imread(imgpath)
    h, w = img.shape[:2]
    ar = w/h; size = w*h
    dia = (w**2 + h**2)**0.5
    wfilesize = filesize / size

    # convert image from BGR color model to HSV color model
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # get average H, S, V (hue, saturation, and value, which represents brightness)
    hsvH, hsvS, hsvV = cv2.split(hsv)
    mean_hsvH = np.mean(hsvH.flatten())
    mean_hsvS = np.mean(hsvS.flatten())
    mean_hsvV = np.mean(hsvV.flatten())

    # create a list that has all the values I want to store
    wlist = [imgname, filesize, h, w, ar, wfilesize, mean_hsvH, mean_hsvS, mean_hsvV]
    print(wlist)
    # transform all the elements in this list to string format
    slist = [str(x) for x in wlist]
    print(slist)
    # combine all the elements in this list and tranform them into a text string
    savelist = "\t".join(slist)
    resultpath = os.path.join('/content/drive/MyDrive', "Final Project", "bbcnews aesthetics results.txt")

    # open a file for writing, and save the results
    with open(resultpath, 'a') as file:
        file.write(savelist + '\n')

In [ ]:
# Img clustering
# download and examine the pre-trained model

from google.colab import drive
import cv2, os, joblib
import numpy as np
import keras.utils as image
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input

# check the pre-trained model
base_model = VGG16(weights="imagenet", include_top=True)
print(base_model.summary())

# build the pre-trained model to use
feature_model = Model(base_model.input, base_model.get_layer('fc1').output)
img_size = (224, 224)
print(feature_model.summary())

In [ ]:
# feature extraction with the pre-trained model for each image

# connect to google drive
drive.mount('/content/drive')
import numpy as np

# read .txt from google drive
imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
print(imgnamefilepath)

imgnamefile = open(imgnamefilepath, "r")

# enumerate each line and extract features
for i, line in enumerate(imgnamefile.readlines()[:]):
    # here I only run the first 10 files. delete 10 if you are running the entire dataset
    imgname = line.rstrip("\r\n")
    print(i, imgname)

    # find the path to my image
    imgpath = os.path.join('/content/drive/MyDrive',"Final Project", "bbcnews", imgname)

    # find how large the image file is
    filesize = os.path.getsize(imgpath)
    print(filesize)

    # this is the folder I store all my extracted features for images
    exfolder = os.path.join('/content/drive/MyDrive',"Final Project", "img exfeature", "")

    # preprocess the image and extract features with this pre-trained model
    img = image.load_img(imgpath, target_size=img_size) # read the image
    image_array = image.img_to_array(img) # convert the image to a numpy array

    image_expand = np.expand_dims(image_array, 0)
    x_train = preprocess_input(image_expand) # normalize the image to 0-1 range
    features_x = feature_model.predict(x_train) # extract features for each image
    print(features_x)
    print(np.shape(features_x)) # print out the shape of extracted features

    # save extracted features
    featuresavepath = os.path.join(exfolder, imgname)
    joblib.dump(features_x[0], featuresavepath)
    print("--SUCCESS--"*10)

print("DONE"*20)

In [ ]:
# combine all the extracted features into one large file

imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
imgnamefile = open(imgnamefilepath, "r")

# combine all the extracted features into one
features_array = []

exfolder = os.path.join('/content/drive/MyDrive',"Final Project", "img exfeature", "")

for lineindex, line in enumerate(imgnamefile.readlines()[:]):
    imgname = line.rstrip('\r\n')
    ex_feature_path = os.path.join(exfolder, imgname)
    imgexfeatures = joblib.load(ex_feature_path)
    flatten_features = np.ndarray.flatten(imgexfeatures)
    # print(flatten_features)
    features_array.append(flatten_features)

# save the file
features_array = np.array(features_array)
print(features_array)
features_savepath = os.path.join(exfolder, "features combine.dat")
joblib.dump(features_array,features_savepath)
print("DONE"*20)

In [ ]:
# inspected our combined file of extracted features

exfolder = os.path.join('/content/drive/MyDrive',"Final Project", "img exfeature", "")
features_savepath = os.path.join(exfolder, "features combine.dat")
features_array = joblib.load(features_savepath)
features_array = np.array(features_array)
print(features_array)
print(np.shape(features_array))


In [ ]:
# conduct PCA to the extracted features

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
imgnamefile = open(imgnamefilepath, "r")

features_savepath = os.path.join(exfolder, "features combine.dat")
features_array = joblib.load(features_savepath)
x = pd.DataFrame(features_array)
print(x)

# standardizing the features
x = StandardScaler().fit_transform(x)

# PCA
pca = PCA(0.95) #95% of the variance is retained.
pca.fit(x)
x_pca = pca.transform(x)

# save PCA
savefolder = os.path.join('/content/drive/MyDrive',"Final Project", "img PCA")
# os.makedirs(savefolder, exist_ok=True)
pca_savepath = os.path.join(savefolder, 'PCA.dat')
joblib.dump(x_pca, pca_savepath)

# save components
components = pca.components_
savepath = os.path.join(savefolder, 'components.txt')
components = pd.DataFrame(components)
components.to_csv(savepath, header=None, index=None, sep='\t', mode='a')

# save variance explained ratio
variance = pca.explained_variance_ratio_
savepath = os.path.join(savefolder, 'variance.txt')
variance = pd.DataFrame(variance)
variance.to_csv(savepath, header=None, index=None, sep='\t', mode='a')

print("DONE"*20)

In [ ]:
# inspected our combined file of extracted features after PCA

features_savepath = os.path.join('/content/drive/MyDrive',"Final Project", "img PCA", "PCA.dat")
features_array = joblib.load(features_savepath)
features_array = np.array(features_array)
print(features_array)
print(np.shape(features_array))

In [ ]:
# run kmeans clustering

from sklearn.cluster import KMeans

#imgnamefilepath = os.path.join('/content/drive/MyDrive',"File4900", "Image clustering", "img all filename.txt")
#imgnamefile = open(imgnamefilepath, "r")

features_savepath = os.path.join('/content/drive/MyDrive',"Final Project", "img PCA", "PCA.dat")
features_array = joblib.load(features_savepath)
X = pd.DataFrame(features_array)
print(X)

# we cluster on the first 200 components
nd = 200
X = X.iloc[:,0:nd]

# run k-means varying the number of clusters from 5 to 11
for K in range(5, 11):
    print('number of cluster', K)
    cl = KMeans(K, random_state=0)
    cl.fit(X)
    labels = cl.labels_

    # merge the k-means results with image names
    imgnamefilepath = os.path.join('/content/drive/MyDrive',"Final Project", "bbc img filenames.txt")
    df = pd.read_csv(imgnamefilepath, sep ='\t', header = None)
    df.columns = ['imgname']
    df['label'] = labels
    print(df)

    # save the labels
    filepath = os.path.join('/content/drive/MyDrive', 'Final Project', 'image clustering', 'img cluster', str(K))
    os.makedirs(filepath, exist_ok=True)
    filepath = os.path.join(filepath, 'label.csv')
    df.to_csv(filepath, index = None, header = None, sep = '\t')
    print("--SUCCESS--"*10)

print("DONE"*50)

In [ ]:
# randomly select images from each cluster so we can inspect the theme of each cluster

import random, shutil
import pandas as pd

# this function copy one image to another folder
def save_img(random_id, imgname, label, K):
    imgpath1 = os.path.join('/content/drive/MyDrive', 'Final Project', 'bbcnews',imgname)
    imgfolder = os.path.join('/content/drive/MyDrive', 'Final Project', 'image clustering', 'img cluster', str(K), str(label), "")
    os.makedirs(imgfolder, exist_ok=True)
    newimgname = str(random_id)+' '+imgname
    imgpath2 = os.path.join(imgfolder, newimgname)
    shutil.copy(imgpath1, imgpath2)

for K in range(5, 11):
    print('number of cluster',K)

    filepath = os.path.join('/content/drive/MyDrive', 'Final Project', 'image clustering', 'img cluster', str(K), 'label.csv')
    df = pd.read_csv(filepath, sep ='\t', header = None)
    df.columns = ['imgname',"label"]

    dr = df.sample(frac=1, random_state=42) # random shuffle images
    dr['random_id'] = np.arange(len(dr)) # add random id

    for labelK in range(0, K):
        dc = dr.loc[dr['label'] == labelK] # select images from each cluster
        dc = dc.iloc[:20,:] # select 20 images from each cluster
        dc.apply(lambda row: save_img(row['random_id'],row['imgname'],row['label'], K),axis=1)

    print("--SUCESS--"*10)
print("DONE"*20)

In [ ]:
# visualize each cluster in a large image grid

import glob
from PIL import Image, ImageDraw, ImageFont
from natsort import natsorted

def fill_square(im, tbw):
    size = (tbw, tbw)
    bg = Image.new('RGB', size, "white")  # create a background image
    im.thumbnail(size, Image.ANTIALIAS)
    w, h = im.size
    bg.paste(im, (int((size[0]-w)/2), int((size[1]-h)/2)))
    return(bg)

w_sq = 300 # width/height of image
w_clusterid = 60 # width of text for cluster id
h_imgid = 60 # height of gap
# fntcluster = ImageFont.truetype('/Library/Fonts/Arial.ttf', 60)

for K in range(5, 11):
    ncol = 20 # how many images to show in each row
    nrow = 1
    nimg = ncol * nrow # total number of image in each cluster to show
    large_w = w_sq*ncol + w_clusterid # total width of the entire image
    large_h = (w_sq + h_imgid) * nrow * K # total height of the entire image
    large = Image.new('RGB', (large_w, large_h), "white")

    for label in range(0, K):
        print('-'* 10, K, label)
        imgpathfolder = os.path.join('/content/drive/MyDrive', 'Final Project', 'image clustering', 'img cluster', str(K), str(label),'')
        imgfiles = glob.glob(imgpathfolder + "*.jpg")
        imgfiles = natsorted(imgfiles)[:nimg]

        d = ImageDraw.Draw(large)
        d.text((0, label * nrow * (w_sq + h_imgid) + 15), str(label+1), fill=(0, 0, 0))

        for j, imgfile in enumerate(imgfiles):
            jw = j%ncol; jh = j//ncol
            img = Image.open(imgfile)
            im_sq = fill_square(img, w_sq)
            im_sq_x = jw*w_sq + w_clusterid
            im_sq_y = h_imgid + jh* (w_sq + h_imgid) + label * nrow * (w_sq + h_imgid)
            large.paste(im_sq, (im_sq_x, im_sq_y))

    large_savepath = os.path.join('/content/drive/MyDrive', 'Final Project', 'img grid', str(K)+".png")
    large.save(large_savepath)

print("DONE"*20)